In [150]:
import csv
import time
import re
import string
import requests
import pytz
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import ceil
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
from email.utils import parsedate_tz
from email.utils import mktime_tz
from sqlalchemy import Column, Integer, DateTime, Date, Interval, PickleType, String, select, func, create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

In [159]:
# Copyright (c) 2018 Sergio Lira <sergio.lira@gmail.com>
#
# MIT Licence. See http://opensource.org/licenses/MIT
#
# Created on 2018-12-03
#

engine = create_engine('sqlite://', echo=False)
Base = declarative_base(bind=engine)

class TimeEntry(Base):
    __tablename__ = 'time_entry'
    id = Column(Integer, primary_key=True)
    date_time = Column(DateTime)
    concepts = Column(PickleType)   
        
    def __init__(self, date_time, concepts):
        self.date_time = date_time
        self.concepts = concepts
        
class TimeSpent(Base):
    __tablename__ = 'time_spent'
    id = Column(Integer, primary_key=True)
    date = Column(Date)
    sdate = Column(String)
    year = Column(Integer)
    month = Column(Integer)
    day = Column(Integer)
    weekday = Column(Integer)
    time_delta = Column(Integer)
    concept = Column(String)
    
    def __init__(self, date, concept, time_delta):
        self.date = date
        self.sdate = date.strftime("%Y-%m-%d")
        self.month = int(date.strftime('%m'))
        self.day = int(date.strftime('%d'))
        self.year = int(date.strftime('%Y'))
        self.weekday = int(date.weekday())
        self.concept = concept
        self.time_delta = time_delta

Base.metadata.create_all()
Session = sessionmaker(bind=engine)

class TimeTracker:
    
    def __init__(self, content, n_days=15, n_months = 3, censor_concepts = False):
        self._content = content
        self.n_days = n_days
        self.n_months = n_months
        self._censor_concepts = censor_concepts
        self._db = Session()

    def load_content(self):
        """
        Initializes the TimeTracker database 
        If the content is of file type txt it initializes by parsing the file else by reading each row in content.
        returns row count added
        """
        #Set look back date and cut_date from max between n_days and n_months        
        look_back = max(self.n_days//31, self.n_months)
        self._cut_date = datetime.today() - relativedelta(months=look_back)
        self._cut_date = date(self._cut_date.date().year, self._cut_date.date().month, 1)
        
        if self._content.endswith('.txt'):
            return self._load_fromm_file()
        else:
            return self._load_fromm_file()
        
        
    def _extract_values_from_row(self, row):
        #Clean date row, format and extract date fields
        #row[0] = row[0].replace(",","").replace("at","")
        date_time= datetime.fromtimestamp(mktime_tz(parsedate_tz(row[0])))
        date_time = date_time.astimezone(pytz.utc)
        #Extract question and score
        concepts = row[1].strip().split(';')
        if self._censor_concepts:
            censored_concepts = []
            for concept in concepts:
                concept = ''.join(['*' if i%3==0 else ch for i, ch in enumerate(concept)])
                censored_concepts.append(concept)
            concepts = censored_concepts
        return TimeEntry(date_time, concepts)
        
    def _load_fromm_file(self):
        with open(self._content, "r") as txtfile:        
            reader = csv.reader(txtfile, delimiter='|', quotechar='"')    
            row_count = 0
            for row in reader:
                #Skip any row that does not matcht he format, including empty rows.
                if len(row) == 2:
                    #Clean date row, format and extract date fields
                    time_entry = self._extract_values_from_row(row)
                    #Only insert date if it will be used
                    if time_entry.date_time.date() >= self._cut_date:
                        #Write to in-memory database
                        self._db.add(time_entry)
                        self._db.commit()
                    row_count += 1
        return row_count
    
    def _calculate_time(self):
        entries = [entry for entry in self._db.query(TimeEntry).order_by(TimeEntry.date_time)]
        if len(entries) > 0 :
            last_entry = entries[0]
            for current_entry in entries[1:]:
                time_delta = current_entry.date_time - last_entry.date_time
                for concept in last_entry.concepts:
                    #print('{} {}: {}'.format(current_entry.sdate, concept, time_delta))
                    time_spent = TimeSpent(current_entry.date_time.date(),
                                           concept.strip(),
                                           time_delta.total_seconds())
                    self._db.add(time_spent)
                    self._db.commit()
                last_entry = current_entry
    
    def _get_last_n_days(self, n_days):
        return self._db.query(TimeSpent.date)\
                            .distinct(TimeSpent.date)\
                            .order_by(TimeSpent.date.desc())\
                            .limit(n_days)\
                            .subquery()
    
    def _get_sum_by_concept_last_n_days(self, n_days):
        last_n_days = self._get_last_n_days(n_days)        
        return self._db.query(TimeSpent.date,
                              TimeSpent.weekday,
                              TimeSpent.concept,
                              func.sum(TimeSpent.time_delta).label('sum_time'))\
                              .join(last_n_days, TimeSpent.date == last_n_days.c.date)\
                              .group_by(TimeSpent.date, TimeSpent.weekday, TimeSpent.concept).subquery()

    def get_last_n_days(self, n_days):
        return self._db.query(self._get_sum_by_concept_last_n_days(n_days)).all()
    
    def get_average_by_concept(self, n_days):
        last_n_days = self._get_sum_by_concept_last_n_days(n_days)
        subquery = self._db.query(last_n_days.c.concept,
                                  func.avg(last_n_days.c.sum_time).label('average_time'))\
                                 .group_by(last_n_days.c.concept).subquery()
        return self._db.query(subquery).order_by(subquery.c.average_time.desc()).all()
    
    def get_average_by_concept_and_day(self, n_days):
        last_n_days = self._get_sum_by_concept_last_n_days(n_days)
        subquery = self._db.query(last_n_days.c.weekday,
                                  last_n_days.c.concept,
                                  func.avg(last_n_days.c.sum_time).label('average_time'))\
                                 .group_by(last_n_days.c.concept, last_n_days.c.weekday).subquery()
        return self._db.query(subquery).order_by(subquery.c.weekday.asc(), subquery.c.average_time.desc()).all()
        

In [160]:
log_file = '/Users/sergio/Dropbox/Reference/Automation/LogTime.txt'
tt = TimeTracker(log_file, censor_concepts = True)
rows_loaded = tt.load_content()
print('Loaded {} time entry rows, inserted {}.'.format(rows_loaded, tt._db.query(TimeEntry.id).count()))
#for entry in tt._db.query(TimeEntry):
   #print(entry.id, entry.date_time, entry.sdate, entry.month, entry.day, entry.year, entry.concepts)
tt._calculate_time()
print('Found {} time spent entries.'.format(tt._db.query(TimeSpent.id).count()))

Loaded 818 time entry rows, inserted 818.
Found 862 time spent entries.


In [161]:
print('\nConcept totals:\n')
print(tt._db.query(TimeSpent.concept, 
                   func.sum(TimeSpent.time_delta))\
                       .group_by(TimeSpent.concept)\
                       .order_by(func.sum(TimeSpent.time_delta).desc()).all())
print('\nLast N days:\n')
print(tt.get_last_n_days(28))
print('\nAverage by concept:\n')
print(tt.get_average_by_concept(28))
print('\nAverage by day:\n')
print(tt.get_average_by_concept_and_day(28))


Concept totals:

[('*le*pi*g', 941801), ('*or*in*', 396155), ('*ea*in* -*Fi*ti*n', 228901), ('*el*-C*re', 172062), ('*ho*es', 155646), ('*am*', 152680), ('*ro*ec*', 149695), ('*nt*rn*t *ed*a', 148044), ('*ni*e', 132996), ('*om*ut*ng', 109620), ('*at*ng', 108505), ('*dm*ni*tr*ti*n', 85159), ('*oc*al*zi*g', 43139), ('*ed*ta*in*', 40555), ('*ea*in* -*No*Fi*ti*n', 27829), ('*ap*ne*e', 26747), ('*ov*', 24771), ('*ig*re*', 22854), ('*on*ex* S*it*h', 21534), ('*tu*yi*g', 20815), ('*el*xi*g', 20234), ('*ov*es*or*Se*ie*', 13705), ('*ud*ob*ok* o* P*dc*st*', 11394), ('*an*a', 10590), ('*oc*or* a*d *ea*th', 9012), ('*ho*pi*g', 8415), ('*xe*ci*in*', 2700)]

Last N days:

[(datetime.date(2019, 1, 20), 6, '*am*', 17804), (datetime.date(2019, 1, 20), 6, '*at*ng', 1088), (datetime.date(2019, 1, 20), 6, '*dm*ni*tr*ti*n', 1797), (datetime.date(2019, 1, 20), 6, '*ea*in* -*No*Fi*ti*n', 5351), (datetime.date(2019, 1, 20), 6, '*el*-C*re', 10240), (datetime.date(2019, 1, 20), 6, '*ho*es', 10039), (datetime.d

https://www.digitalocean.com/community/tutorials/data-analysis-and-visualization-with-pandas-and-jupyter-notebook-in-python-3
http://pbpython.com/simple-graphing-pandas.html